<a href="https://colab.research.google.com/github/sajini1madushika/medi-predict-web-for-public-hospitals/blob/main/mediPredict_api_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok python-multipart joblib pandas


In [2]:
api_code = r"""
# ==============================
# MediPredict-SL Unified API
# ==============================

import uuid, sqlite3, json, os
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from datetime import datetime

try:
    import joblib, pandas as pd
except:
    joblib, pd = None, None

app = FastAPI(title="MediPredict-SL API")

# ===== Database =====
DB = "patients.db"
conn = sqlite3.connect(DB, check_same_thread=False)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS patients (id TEXT PRIMARY KEY, name TEXT, age INTEGER, gender TEXT, created_at TEXT)")
cursor.execute("CREATE TABLE IF NOT EXISTS history (patient_id TEXT, report_json TEXT, created_at TEXT)")
conn.commit()

# ===== ML Model (optional) =====
ML_MODEL_PATH = "xgb_chronic_disease_model.pkl"
FEATURES_PATH = "feature_names.json"
ml_model, feature_names = None, []
if joblib and os.path.exists(ML_MODEL_PATH) and os.path.exists(FEATURES_PATH):
    try:
        ml_model = joblib.load(ML_MODEL_PATH)
        with open(FEATURES_PATH, "r") as f:
            feature_names = json.load(f)
    except:
        ml_model = None

# ===== Schemas =====
class RegisterRequest(BaseModel):
    name: str
    age: int
    gender: str

class LabReport(BaseModel):
    patient_id: str
    fasting_glucose: float = None
    random_glucose: float = None
    hba1c: float = None
    systolic: float = None
    diastolic: float = None
    total_chol: float = None
    ldl: float = None
    hdl: float = None
    trig: float = None
    creatinine: float = None
    urea: float = None
    egfr: float = None
    ast: float = None
    alt: float = None
    tbil: float = None
    notes: str = None

# ===== Root & Health =====
@app.get("/")
def root(): return {"msg": "MediPredict API running"}
@app.get("/health")
def health(): return {"ok": True, "ml_loaded": ml_model is not None}

# ===== Auth (mock) =====
@app.post("/auth/login")
def login(payload: dict):
    u, p = payload.get("username"), payload.get("password")
    if u in ["doctor","pharmacist","patient","trainee"] and p=="pass":
        return {"access_token":"fake-jwt","role":u,
                "user":{"id":f"{u[:1].upper()}-1001","name":u.title(),"email":f"{u}@mediPredict.local"}}
    return {"error":"Invalid credentials"}

# ===== Patients =====
@app.post("/register")
def register(data: RegisterRequest):
    pid = "P-" + str(uuid.uuid4())[:8]
    cursor.execute("INSERT INTO patients VALUES (?,?,?,?,?)",(pid,data.name,data.age,data.gender,datetime.utcnow().isoformat()))
    conn.commit()
    return {"patient_id": pid, "qrCode": pid}

@app.get("/patients")
def get_patients():
    cursor.execute("SELECT id,name,age,gender FROM patients")
    return [{"id":r[0],"name":r[1],"age":r[2],"gender":r[3],"phone":"+94 71 000 0000",
             "conditions":["Diabetes"],"latestRiskScore":0.5} for r in cursor.fetchall()]

@app.get("/patients/{pid}")
def get_patient(pid: str):
    cursor.execute("SELECT id,name,age,gender FROM patients WHERE id=?",(pid,))
    r = cursor.fetchone()
    if not r: return {"error":"Patient not found"}
    return {"id":r[0],"name":r[1],"age":r[2],"gender":r[3],"phone":"+94 71 000 0000",
            "conditions":["Diabetes"],"qrCode":r[0]}

# ===== Visits / History =====
@app.post("/predict_rules")
def predict_rules(report: LabReport):
    flags=[]
    if report.fasting_glucose and report.fasting_glucose>=126: flags.append("Diabetes Risk (FBS high)")
    if report.hba1c and report.hba1c>=6.5: flags.append("Diabetes Risk (HbA1c high)")
    if report.systolic and report.systolic>=140: flags.append("Hypertension Risk (Systolic high)")
    if report.diastolic and report.diastolic>=90: flags.append("Hypertension Risk (Diastolic high)")
    if report.total_chol and report.total_chol>=240: flags.append("Cholesterol Risk (High TC)")
    if report.ldl and report.ldl>=160: flags.append("Cholesterol Risk (High LDL)")
    if report.egfr and report.egfr<60: flags.append("CKD Risk (Low eGFR)")
    if report.creatinine and report.creatinine>1.2: flags.append("Kidney Risk (High Creatinine)")
    if report.ast and report.ast>40: flags.append("Liver Risk (AST high)")
    if report.alt and report.alt>40: flags.append("Liver Risk (ALT high)")
    if report.tbil and report.tbil>1.2: flags.append("Liver Risk (High Bilirubin)")
    cursor.execute("INSERT INTO history VALUES (?,?,?)",(report.patient_id,json.dumps(report.dict()),datetime.utcnow().isoformat()))
    conn.commit()
    return {"patient_id":report.patient_id,"flags":flags,"timestamp":datetime.utcnow().isoformat()}

@app.get("/history/{pid}")
def get_history(pid: str):
    cursor.execute("SELECT report_json,created_at FROM history WHERE patient_id=?",(pid,))
    return {"patient_id":pid,"history":[{"report":json.loads(r),"date":d} for r,d in cursor.fetchall()]}

@app.get("/patients/{pid}/visits")
def get_visits(pid: str):
    cursor.execute("SELECT report_json,created_at FROM history WHERE patient_id=?",(pid,))
    rows=cursor.fetchall()
    return [{"id":f"V-{i+1}","date":d[:10],"doctor":"Dr. Demo","treatment":"Metformin 500mg","riskScore":0.7,
             "diabetes":1,"hypertension":1,"cholesterol":0} for i,(r,d) in enumerate(rows)]

# ===== Prescriptions =====
@app.get("/patients/{pid}/prescriptions")
def get_prescriptions(pid: str):
    return [{"id":"RX-501","drug":"Atorvastatin","dose":"10mg","status":"Pending"},
            {"id":"RX-502","drug":"Metformin","dose":"500mg","status":"Issued"}]

@app.put("/prescriptions/{rxid}")
def update_prescription(rxid: str, payload: dict):
    return {"id":rxid,"status":payload.get("status","Pending")}

# ===== QR Scan =====
@app.get("/scan/{pid}")
def scan_qr(pid: str): return {"pid":pid,"target":f"/doctor/{pid}"}

# ===== Reports =====
@app.post("/patients/{pid}/reports")
def upload_report(pid: str, file: UploadFile = File(...)):
    content=file.file.read()
    fname=f"report_{pid}_{file.filename}"
    with open(fname,"wb") as f: f.write(content)
    return {"patient_id":pid,"filename":fname,"status":"Uploaded"}

@app.get("/patients/{pid}/reports")
def get_reports(pid: str):
    return [{"filename":"lab_feb2025.pdf","uploaded":"2025-02-20"},
            {"filename":"ecg_apr2025.png","uploaded":"2025-04-15"}]

# ===== ML Prediction =====
@app.post("/predict_ml")
def predict_ml(report: LabReport):
    if not ml_model: return {"error":"ML model not loaded"}
    row={f:getattr(report,f,0) or 0 for f in feature_names}
    df=pd.DataFrame([row])
    pred=int(ml_model.predict(df)[0])
    prob=float(ml_model.predict_proba(df)[0][1])
    return {"prediction":pred,"probability":round(prob,3)}
"""

with open("mediPredict_api.py", "w") as f:
    f.write(api_code)
print("✅ mediPredict_api.py created")


✅ mediPredict_api.py created


In [4]:
from pyngrok import ngrok

# Paste your token here
ngrok.set_auth_token("33FWiTdaE5gCJ0nchcNy2UalKAi_3MkjUUusuWJUSp6B1E2Jd")


In [5]:
import nest_asyncio, uvicorn, threading

nest_asyncio.apply()

def run():
    uvicorn.run("mediPredict_api:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run, daemon=True)
thread.start()

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)


INFO:     Started server process [389]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🌍 Public URL: NgrokTunnel: "https://reemergent-exultantly-felecia.ngrok-free.dev" -> "http://localhost:8000"


In [6]:
import requests

BASE = "https://reemergent-exultantly-felecia.ngrok-free.dev"

# Health check
print("Health:", requests.get(BASE + "/health").json())

# Register a patient
reg = requests.post(BASE + "/register", json={"name":"Test Patient","age":45,"gender":"M"}).json()
print("Register:", reg)

pid = reg["patient_id"]

# Predict rules
payload = {"patient_id": pid, "fasting_glucose": 145, "hba1c": 7.2, "systolic": 150}
print("Predict rules:", requests.post(BASE + "/predict_rules", json=payload).json())

# History
print("History:", requests.get(BASE + f"/history/{pid}").json())


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3496511139.py", lin

INFO:     34.42.141.38:0 - "GET /health HTTP/1.1" 200 OK
Health: {'ok': True, 'ml_loaded': False}
INFO:     34.42.141.38:0 - "POST /register HTTP/1.1" 200 OK
Register: {'patient_id': 'P-1e823d03', 'qrCode': 'P-1e823d03'}
INFO:     34.42.141.38:0 - "POST /predict_rules HTTP/1.1" 200 OK
Predict rules: {'patient_id': 'P-1e823d03', 'flags': ['Diabetes Risk (FBS high)', 'Diabetes Risk (HbA1c high)', 'Hypertension Risk (Systolic high)'], 'timestamp': '2025-09-26T19:13:29.885257'}
INFO:     34.42.141.38:0 - "GET /history/P-1e823d03 HTTP/1.1" 200 OK
History: {'patient_id': 'P-1e823d03', 'history': [{'report': {'patient_id': 'P-1e823d03', 'fasting_glucose': 145.0, 'random_glucose': None, 'hba1c': 7.2, 'systolic': 150.0, 'diastolic': None, 'total_chol': None, 'ldl': None, 'hdl': None, 'trig': None, 'creatinine': None, 'urea': None, 'egfr': None, 'ast': None, 'alt': None, 'tbil': None, 'notes': None}, 'date': '2025-09-26T19:13:29.867027'}]}
